In [3]:
import json

In [4]:
root_location = '/home/a/Desktop/MSP-Podcast/pipeline/INPUTS_OUTPUTS/Outputs/'


with open(os.path.join(os.path.join(root_location, "Short_files.json")), "r") as openfile:
    audio_files = json.load(openfile)

In [5]:
for filename, f_info  in audio_files.items():
    break

In [9]:
f_path = f_info['filepaths']['opensmile_lld']

In [15]:
def TryToFloat(single_data):
    try:
        return float(single_data)
    except:
        return None
    
def LoadFeature(filename):
    content = open(filename, 'r').read()
    data = content.split('@data\n')[1].split('\n')
    data = filter(None, data)
    feature = [[TryToFloat(data_split) for data_split in d.split(',') \
                if TryToFloat(data_split)!=None] for d in data]
    return np.array(feature)

In [16]:
LoadFeature(f_path)

array([[ 0.000000e+00,  2.098476e+02,  7.457917e-01, ..., -5.050770e-01,
        -2.337668e+00, -1.067880e-01],
       [ 1.600000e-02,  2.485446e+02,  7.827879e-01, ..., -9.466939e-01,
        -3.285222e+00, -7.556780e-02],
       [ 3.200000e-02,  2.854767e+02,  8.175290e-01, ..., -2.302814e+00,
        -3.532829e+00, -7.100361e-01],
       ...,
       [ 6.608000e+00,  0.000000e+00,  0.000000e+00, ..., -5.839880e+00,
        -3.835547e+00,  1.459241e+00],
       [ 6.624000e+00,  0.000000e+00,  0.000000e+00, ..., -4.503720e+00,
        -2.473446e+00,  1.987281e+00],
       [ 6.624000e+00,  0.000000e+00,  0.000000e+00, ..., -2.556796e+00,
        -1.113743e+00,  1.123780e+00]])

In [18]:
import numpy as np
from keras.models import Model
from keras.layers.core import Dense, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers import SimpleRNN, LSTM, Lambda, Input, Dot, Concatenate
import tensorflow as tf
import random
import os
from keras import backend as K
from scipy.io import loadmat
import csv
import argparse
# Ignore warnings & Fix random seed
import warnings
warnings.filterwarnings("ignore")
random.seed(999)
random_seed=99
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from tqdm import tqdm

In [2]:
def mean():  
    def func(x):
        return K.mean(x, axis=1, keepdims=False)
    return Lambda(func)

def reshape_1():
    def func(x):
        feat_num = 130
        chunk_num = 11
        return K.reshape(x, (1, chunk_num, feat_num))
    return Lambda(func)

def reshape_2():
    def func(x):
        feat_num = 130
        chunk_num = 17
        return K.reshape(x, (1, chunk_num, feat_num))
    return Lambda(func)

# Split Original batch Data into Small-Chunk batch Data Structure with different step window size
def DiffRslChunkSplitTestingData(Batch_data, chunk_num):
    """
    Note!!! This function can't process sequence length which less than given chunk_size (i.e.,1sec=62frames)
    Please make sure all your input data's length are greater then given chunk_size
    """
    chunk_size = 62  # (62-frames*0.016sec) = 0.992sec
    n = 1
    num_shifts = n*chunk_num-1  # max_length = 11sec, chunk needs to shift 10 times to obtain total 11 chunks for each utterance
    Split_Data = []
    for i in range(len(Batch_data)):
        data = Batch_data[i]
        # Shifting-Window size varied by differenct length of input utterance => Different Resolution
        step_size = int(int(len(data)-chunk_size)/num_shifts)      
        # Calculate index of chunks
        start_idx = [0]
        end_idx = [chunk_size]
        for iii in range(num_shifts):
            start_idx.extend([start_idx[0] + (iii+1)*step_size])
            end_idx.extend([end_idx[0] + (iii+1)*step_size])    
        # Output Split Data
        for iii in range(len(start_idx)):
            Split_Data.append( data[start_idx[iii]: end_idx[iii]] )    
    return np.array(Split_Data)
###############################################################################

In [20]:
f_path

array('/home/a/Desktop/MSP-Podcast/pipeline/INPUTS_OUTPUTS/Outputs/Short_split_OpenSmile_lld_IS13ComParE/MSP-PODCAST_5198_0001.arff',
      dtype='<U124')

In [26]:
print("Music/speech inference: ")
for filename, f_info in tqdm(audio_files.items()):
    f_path = f_info['filepaths']['opensmile_lld']
    audio = LoadFeature(f_path)#[:,1:].astype(np.float32)
    break

  0%|          | 0/499 [00:00<?, ?it/s]

Music/speech inference: 


In [27]:
audio.shape

(416, 131)